In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import matplotlib.pyplot as plt

# internal imports
from mld_dynamics import mld, h, x_min, x_max, fc_min, fc_max, d, l
from warm_start_hmpc.controller import HybridModelPredictiveController
from warm_start_hmpc.mcais import solve_dare, mcais
from visualizer import vis, animate

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [2]:
# controller horizon
T = 20

# weight matrices
Q = np.eye(mld.nx) * h
R = np.vstack([1.]+[0.]*(mld.nu - 1)).T * h

# LQR terminal cost
Bu = mld.B[:, :1]
Ru = R[:, :1]
P, K = solve_dare(mld.A, Bu, Q.dot(Q), Ru.dot(Ru))
Q_T = np.linalg.cholesky(P).T
objective = [Q, R, Q_T]

# MCAIS terminal set
A_cl = mld.A + Bu.dot(K)
lhs = mld.F + mld.G[:, :1].dot(K)
rhs = mld.h
terminal_set = mcais(A_cl, lhs, rhs, verbose=True)

# hybrid controller
controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

Using license file /Users/tobia/codes/gurobi.lic
Academic license - for non-commercial use only
Time horizon: 34. Convergence index: 0.000000. Number of facets: 162.
Maximal constraint-admissible invariant set found. Removing redundant facets ... minimal facets are 102.


In [3]:
# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

## Solve in closed loop

## Statistical Analysis

In [4]:
e_sd = 0.# 0.001, 0.003, 0.01, 0.03
N_sim = 5#50
N_samples = 1#100
i = 0

solver_times = []
solver_times_ws = []
solver_times_gurobi = []
errors = []

while len(solver_times) < N_samples:
    np.random.seed(i)

    rollout_success = True

    solver_times_i = []
    solver_times_ws_i = []
    solver_times_gurobi_i = []
    errors_i = []

    x_sim = [x0]
    ws = None

    for t in range(N_sim):
        print((i,t), end='\r')

#             # project previous state in feasible set
#             x_prev = np.minimum(np.maximum(x_sim[-1], x_min), x_max)

        # solve with gurobi
        x_gurobi, cost_gurobi, solver_time_gurobi = controller.feedforward_gurobi(
            x_sim[-1],
            {'OutputFlag': 0}
        )
        solver_times_gurobi_i.append(solver_time_gurobi)

        # solve without warm start
        solution, leaves, solver_time = controller.feedforward(
            x_sim[-1],
            printing_period=None
        )
        solver_times_i.append(solver_time)

        # solve with warm start
        solution_ws, leaves_ws, solver_time_ws = controller.feedforward(
            x_sim[-1],
            warm_start=ws,
            printing_period=None
        )
        solver_times_ws_i.append(solver_time_ws)

        # check feasibility
        if solution is None:
            break
        assert np.isclose(solution.objective, solution_ws.objective)
#             assert np.isclose(solution.objective, cost_gurobi)

        # generate random error of specified norm
        e_t = e_sd * np.multiply(np.random.randn(mld.nx), x_max)
        errors_i.append(e_t)

        # generate warm start
        ws = controller.construct_warm_start(
            leaves_ws,
            solution_ws.variables['x'][0],
            solution.variables['uc'][0],
            solution.variables['ub'][0],
            e_t
        )

        # update state
        x_sim.append(solution_ws.variables['x'][1] + e_t)

        solver_times.append(solver_times_i)
        solver_times_ws.append(solver_times_ws_i)
        solver_times_gurobi.append(solver_times_gurobi_i)
        errors.append(errors_i)

    i += 1

ValueError: too many values to unpack (expected 3)

In [ ]:
solver_times

In [ ]:
solver_times_ws

In [ ]:
solver_times_gurobi